In [3]:
import azure.cognitiveservices.speech as speechsdk
import tkinter as tk
from tkinter import messagebox, Text
from PIL import Image, ImageTk
import joblib
import re
import nltk
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side
import pathlib
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer  # Menggunakan SnowballStemmer untuk bahasa Indonesia
nltk.download('stopwords')

root = tk.Tk()
class App():
    def __init__(self, master):
        self.window = master
        self.window.title("sentimen analisis")
        self.window.resizable(False, False)
        self.bow_vectorizer = joblib.load(r'bow_vectorizer.pkl')

        # Menambahkan latar belakang gambar tanpa zoom
        background_image = Image.open(
            r"page1.png")
        background_resize = background_image.resize((1440, 720))
        background_image = ImageTk.PhotoImage(
            background_resize)  # Ganti dengan path gambar Anda
        background_label = tk.Label(root, image=background_image)
        background_label.place(x=-2, y=-2)

        # Menambahkan kotak teks
        self.text_box = Text(root, height=12, width=55, bg="white",
                        borderwidth=0, font=('Constantia', 12))
        self.text_box.place(x=200, y=130)

        self.text_to_analyze = self.text_box.get("1.0", tk.END)
        self.preprocess_text(self.text_to_analyze)

        image_trash = r'trash.png'
        image_trash = Image.open(image_trash)
        mic_resize = image_trash.resize((50, 50))
        image_trash = ImageTk.PhotoImage(mic_resize)  # Ganti dengan path gambar Anda
        self.clear_button = tk.Button(root, command=self.clear_text, image=image_trash,
                                width=50, height=50, borderwidth=0, highlightthickness=0)
        self.clear_button.place(x=830, y=230)

        # Membuat tombol untuk pengenalan ucapan
        # Replace with the path to your image file
        image_mic = r'mic.png'
        image_mic = Image.open(image_mic)
        mic_resize = image_mic.resize((50, 50))
        image_mic = ImageTk.PhotoImage(mic_resize)
        start_button = tk.Button(root, command=self.recognize_speech, image=image_mic,
                                width=50, height=50, borderwidth=0, highlightthickness=0)
        start_button.place(x=830, y=170)

        # Membuat tombol untuk analisis sentimen
        sentiment_button = tk.Button(root, text="Analisis Sentimen", command=self.analyze_sentiment, font=(
            "Helvetica", 16), bg="#545454", fg="white", borderwidth=0)
        sentiment_button.place(x=830, y=290)
        self.font = Font(bold=True)
        self.border = Border(left=Side(border_style='thin', color='00000000'),
                             right=Side(border_style='thin', color='00000000'),
                             top=Side(border_style='thin', color='00000000'),
                             bottom=Side(border_style='thin', color='00000000'))
        self.alignment = Alignment(horizontal='center', vertical='center')
        self.file = pathlib.Path('data.xlsx')
        if self.file.exists():
            pass
        else:
            self.file = Workbook()
            sheet = self.file.active
            headers = ['Hari', 'Waktu', 'text','Sentimen']

            for col_num, header in enumerate(headers, 1):
                cell = sheet.cell(row=1, column=col_num)
                cell.value = header
            for col_num in range(1, len(headers) + 1):
                cell = sheet.cell(row=1, column=col_num)
                cell.font = self.font
                cell.border = self.border
                cell.alignment = self.alignment
            self.file.save('data.xlsx')

    # Fungsi untuk menghapus teks dari kotak teks
    def clear_text(self):
        self.text_box.delete("1.0", tk.END)

    # Fungsi pengenalan ucapan

    def recognize_speech(self):
        # Setel kunci dan wilayah dari sumber daya Azure Speech Anda
        speech_key = "5544cd8ec2fb442c8527470731b0daae"
        region = "southeastasia"

        # Buat objek konfigurasi dengan kunci dan wilayah
        speech_config = speechsdk.SpeechConfig(
            subscription=speech_key, region=region)

        # Atur bahasa pengenalan ke Bahasa Indonesia
        speech_config.speech_recognition_language = "id-ID"

        # Buat pengenal ucapan dengan konfigurasi
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

        # Mulai pengenalan ucapan
        print("Mulai berbicara...")
        self.result = speech_recognizer.recognize_once()

        # Cetak hasil pengenalan ucapan
        if self.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            self.text_box.insert(tk.END, self.result.text)
        elif self.result.reason == speechsdk.ResultReason.NoMatch:
            messagebox.showinfo("Hasil", "Tidak ada ucapan yang terdeteksi.")
        elif self.result.reason == speechsdk.ResultReason.Canceled:
            cancellation_details = self.result.cancellation_details
            messagebox.showerror("Error", "Pengenalan ucapan dibatalkan: {}".format(
                cancellation_details.reason))
            if cancellation_details.reason == speechsdk.CancellationReason.Error:
                messagebox.showerror("Error", "Kesalahan: {}".format(
                    cancellation_details.error_details))
        self.analyze_sentiment()
    # Fungsi untuk preprocessing teks
    def preprocess_text(self, text):
        # Menghilangkan karakter khusus dan angka
        text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
        # Mengubah teks ke huruf kecil
        text = text.lower()
        # Menghilangkan stop words
        stop_words = set(stopwords.words('indonesian'))
        text = ' '.join([word for word in text.split() if word not in stop_words])
        # Stemming
        stemmer = SnowballStemmer('indonesian')  # Menggunakan SnowballStemmer untuk bahasa Indonesia
        text = ' '.join([stemmer.stem(word) for word in text.split()])
        return text

    def analyze_sentiment(self):
        # Lakukan preprocessing pada teks
        cleaned_text = self.preprocess_text(self.text_to_analyze)

        # Transform the cleaned text to the same format as the training data
        text_to_analyze_transformed = self.bow_vectorizer.transform([cleaned_text])

        # Muat model yang telah disimpan
        try:
            loaded_model = joblib.load(r'C:\Users\rizky\OneDrive\Dokumen\GitHub\test\testpython\pemrosesan text\project pemro\linear1000_model.pkl')
            # Lakukan prediksi menggunakan model yang dimuat
            predicted_sentiment = loaded_model.predict(text_to_analyze_transformed)
            # Tampilkan hasil prediksi dalam messagebox
            messagebox.showinfo("Hasil Analisis Sentimen", "Sentimen teks ini adalah: " + str(predicted_sentiment[0]))
        except Exception as e:
            messagebox.showerror("Error", "Tidak dapat memuat model: " + str(e))

app = App(root)
root.mainloop()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rizky\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TclError: image "pyimage4" doesn't exist

: 

In [1]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import re

class SpeechToTextApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Speech to Text Converter")
        self.master.geometry("800x600")
        self.master.resizable(True, True)

        # Mengatur tema dan gaya
        self.style = ttk.Style(self.master)
        self.style.theme_use('clam')  # Tema 'clam' memiliki tampilan yang bersih dan modern

        # Membuat frame utama
        self.main_frame = ttk.Frame(self.master, padding="10 10 10 10")
        self.main_frame.pack(expand=True, fill='both')

        # Membuat label judul
        self.title_label = ttk.Label(self.main_frame, text="Speech to Text Converter", font=("Helvetica", 18, "bold"))
        self.title_label.pack(pady=20)

        # Membuat area teks untuk menampilkan hasil
        self.text_area = tk.Text(self.main_frame, height=15, width=50)
        self.text_area.pack(pady=20)

        # Membuat tombol untuk memulai pengenalan ucapan
        self.start_button_image = self.load_image(r'C:\Users\rizky\OneDrive\Dokumen\GitHub\test\testpython\pemrosesan text\project pemro\mic.png', (50, 50))
        self.start_button = ttk.Button(self.main_frame, image=self.start_button_image, text="Start", compound=tk.LEFT, command=self.start_speech_to_text)
        self.start_button.pack(side=tk.LEFT, padx=10)

        # Membuat tombol untuk menghapus teks
        self.clear_button_image = self.load_image(r'C:\Users\rizky\OneDrive\Dokumen\GitHub\test\testpython\pemrosesan text\project pemro\trash.png', (50, 50))
        self.clear_button = ttk.Button(self.main_frame, image=self.clear_button_image, text="Clear", compound=tk.LEFT, command=self.clear_text)
        self.clear_button.pack(side=tk.RIGHT, padx=10)

    def load_image(self, path, size):
        image = Image.open(path)
        image = image.resize(size, Image.ANTIALIAS)
        return ImageTk.PhotoImage(image)

    def start_speech_to_text(self):
        # Fungsi untuk memulai pengenalan ucapan
        pass

    def clear_text(self):
        # Fungsi untuk menghapus teks dari area teks
        self.text_area.delete("1.0", tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = SpeechToTextApp(root)
    root.mainloop()

C:\Users\rizky\AppData\Local\Temp\ipykernel_29900\1928799222.py:41: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(size, Image.ANTIALIAS)
